# **샘플 데이터와 유사도 함수 실습**

강의에서 예시로 사용한 데이터를 이용해 유사도 함수를 계산해보겠습니다.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(2021)

# 1. 유클리디안 유사도

유사도를 계산할 유저를 인덱스로 아이템을 컬럼으로 하는 데이터를 정의함\
강의와 같이 유저가 평가하지 않은 아이템에 대해서는 결측값으로 표시됨

1.1 Sample Data

In [2]:
data = [
    [1., None, 1., None],
    [None, 1., 1., None],
    [1, None, 1., 1.]
]

df = pd.DataFrame(
    data=data,
    index=["userA", "userB", "userC"],
    columns=["itemA", "itemB", "itemC", "itemD"]
)

df

,itemA,itemB,itemC,itemD
userA,1.0,NaN,1.0,NaN
userB,NaN,1.0,1.0,NaN
userC,1.0,NaN,1.0,1.0


1.2 결측값 제거

유클리디안 거리 계산시 결측값이 있으면 안되므로, 
결측값을 0으로 대체

In [3]:
df = df.fillna(0)
df

,itemA,itemB,itemC,itemD
userA,1.0,0.0,1.0,0.0
userB,0.0,1.0,1.0,0.0
userC,1.0,0.0,1.0,1.0


1.3 유클리디안 유사도 계산

유저-아이템 평가 행렬에서 유저별로 유클리디안 유사도를 계산


$$
유클리디안 유사도 = \frac{1}{\text{유클리디안 거리} + \text{1e-5}}
$$

`sklearn.metrics.pairwise`의 `euclidean_distances`를 이용해 유클리디안 거리를 계산합니다.

In [4]:
# 유클리디안 거리 계산

from sklearn.metrics.pairwise import euclidean_distances

`euclidean_distances`에 X와 Y를 입력할 경우, X와 Y의 각 Row끼리 유클리디안 거리를 계산합니다.

유저A와 유저B의 Row를 각각 X와 Y로 입력하면 두 유저의 유클리디안 거리를 계산할 수 있습니다.

In [5]:

euclidean_distances(
    X = df.loc[["userA"]],
    Y = df.loc[["userB"]],
)

array([[1.41421356]])

In [6]:
# 전체 데이터를 입력할 경우, 각 row에 기반한 유클리디안 거리를 계산할 수 있음  
euclidean_distances(df)

array([[0.        , 1.41421356, 1.        ],
       [1.41421356, 0.        , 1.73205081],
       [1.        , 1.73205081, 0.        ]])

In [7]:
distance = euclidean_distances(df)
 # 0일때 역수를 취했을때 무한대값이 나오는 것을 방지하기 위해 아주 작은 수를 더하고, 역수를 취해서 similarity 구함
similarity = 1 / (distance + 1e-5) 
similarity 

array([[1.00000000e+05, 7.07101781e-01, 9.99990000e-01],
       [7.07101781e-01, 1.00000000e+05, 5.77346936e-01],
       [9.99990000e-01, 5.77346936e-01, 1.00000000e+05]])

# 2. 코사인 유사도

2.1 코사인 유사도 계산

유클리디안 유사도 계산에 사용한 데이터를 이용해 코사인 유사도를 계산해 보겠습니다.

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

유클리디안 유사도 계산과 마찬가지로 X와 Y를 각각 입력할 수 있고, X만 입력할 수 있습니다.

In [10]:
cosine_similarity(
    X=df.loc[["userA"]],
    Y=df.loc[["userB"]]
)

array([[0.5]])

In [11]:
# 모든 유저 사이의 코사인 유사도를 계산하면 다음과 같음

cosine_similarity(df)

array([[1.        , 0.5       , 0.81649658],
       [0.5       , 1.        , 0.40824829],
       [0.81649658, 0.40824829, 1.        ]])

# 3. 피어슨 유사도

아이템에 대한 유저의 선호도를 반영한 행렬에 대해 피어슨 유사도를 적용해 보겠습니다.

3.1 Sample Data

새로운 데이터를 정의합니다.

In [12]:
data = [
    [4., 5., 4., 3.],
    [3., 4., 3., 2.],
    [4., 4., 5., 3.],
]

df = pd.DataFrame(
    data = data,
    index = ["userA", "userB", "userC"],
    columns=["itemA", "itemB", "itemC", "itemD"]
)

3.2 피어슨 유사도 계산

데이터의 각 row별로 유사도 계산함

In [13]:
np.corrcoef(df)

array([[1. , 1. , 0.5],
       [1. , 1. , 0.5],
       [0.5, 0.5, 1. ]])

3.3 코사인 유사도 계산

피어슨 유사도는 유저 또는 아이템 별로 특성을 제거한 데이터에 코사인 유사도를 적용한 것과 같습니다.

유저별로 선호도 평균을 계산하고, 기존 데이터에서 유저별 선호도를 제거합니다.


1. `df.mean(axis=1)`은 각 행에 대해 평균을 계산합니다.

2. `df1.sub(df2, axis=0)`은 인덱스를 기준으로 두 데이터의 차를 계산합니다.

In [17]:
# 유저 평균계산(유저가 주는 특성)
df.mean(axis=1)

userA    4.0
userB    3.0
userC    4.0
dtype: float64

In [18]:
user_mean = df.mean(axis=1)
# 특성을 정수에서 제거해야함
df_sub = df.sub(user_mean, axis=0)

In [19]:
df_sub

,itemA,itemB,itemC,itemD
userA,0.0,1.0,0.0,-1.0
userB,0.0,1.0,0.0,-1.0
userC,0.0,0.0,1.0,-1.0


In [20]:
cosine_similarity(df_sub) # 피어슨 유사도 값과 동일함

array([[1. , 1. , 0.5],
       [1. , 1. , 0.5],
       [0.5, 0.5, 1. ]])

## 4.자카드 유사도

4.1 Sample Data

유저마다 다른 아이템에 대해 선호도를 평가한 데이터를 정의합니다.

In [21]:
data = [
    [4., 0., 4., 3., 0.],
    [3., 4., 0., 2., 0.],
    [0., 0., 4., 5., 3.],
]

df = pd.DataFrame(
    data=data,
    index=["userA", "userB", "userC"],
    columns=["itemA", "itemB", "itemC", "itemD", "itemE"],
)
df

,itemA,itemB,itemC,itemD,itemE
userA,4.0,0.0,4.0,3.0,0.0
userB,3.0,4.0,0.0,2.0,0.0
userC,0.0,0.0,4.0,5.0,3.0


4.2 자카드 유사도 계산

In [22]:
from sklearn.metrics import jaccard_score

jaccard_score는 값의 크기는 무시하고 아이템의 유무를 0과 1로 표현\
0보다 큰 값을 가지는 경우 선호도를 평가한 것을 표현하기 위해 1로 대체함

In [23]:
df[df>0]=1 # 값이 들어있는지 아닌지에 대한 값만 구별할 수 있도록 변경함
df

,itemA,itemB,itemC,itemD,itemE
userA,1.0,0.0,1.0,1.0,0.0
userB,1.0,1.0,0.0,1.0,0.0
userC,0.0,0.0,1.0,1.0,1.0


In [24]:
# jaccard_score 비교하는 두 유저의 값을 각각 입력해야함!!(데이터 프레임 한번에 적용X)

jaccard_score(
    df.loc["userB"],
    df.loc["userC"],
)

0.2